In [1]:
import requests
import pandas as pd

In [2]:
headers = {
        'Content-Type': 'application/json'
        }
requestResponse = requests.get("https://api.tiingo.com/tiingo/daily/-P-S?token=8a3dcff27b736ecb02df8d6da6db86e3d35354eb",
                                    headers=headers)
print(requestResponse.json())

{'exchangeCode': 'NYSE', 'description': 'PURE RESOURCES INC', 'ticker': '-P-S', 'startDate': '2018-08-22', 'name': 'Prudential Financial Inc', 'endDate': '2020-11-20'}


In [3]:
print(requestResponse.json())

{'exchangeCode': 'NYSE', 'description': 'PURE RESOURCES INC', 'ticker': '-P-S', 'startDate': '2018-08-22', 'name': 'Prudential Financial Inc', 'endDate': '2020-11-20'}


In [4]:
pd.DataFrame.from_dict(requestResponse.json(), orient="index").transpose()

,exchangeCode,description,ticker,startDate,name,endDate
0,NYSE,PURE RESOURCES INC,-P-S,2018-08-22,Prudential Financial Inc,2020-11-20


In [5]:
#https://apimedia.tiingo.com/docs/tiingo/daily/supported_tickers.zip

In [6]:
df = pd.read_csv('supported_tickers.csv')
df.head()

,ticker,exchange,assetType,priceCurrency,startDate,endDate
0,-P-H,NYSE,Stock,USD,2013-03-27,2020-09-22
1,-P-S,NYSE,Stock,USD,2018-08-22,2020-11-20
2,000001,SHE,Stock,CNY,2007-08-30,2020-11-20
3,000002,SHE,Stock,CNY,2000-01-04,2020-11-20
4,000003,SHE,Stock,CNY,NaN,NaN


In [7]:
df[df.priceCurrency=='USD'].exchange.unique()

array(['NYSE', nan, 'NMFQS', 'OTCGREY', 'SHG', 'PINK', 'NYSE ARCA',
       'NASDAQ', 'OTCMKTS', 'OTCBB', 'OTCQB', 'NYSE MKT', 'OTCQX',
       'OTCCE', 'ASX', 'BATS', 'AMEX', 'LSE', 'EXPM', 'CSE'], dtype=object)

In [8]:
#df[(df.priceCurrency=='USD') & (df.exchange =='CSE')]

In [9]:
list_of_strings = ['NYSE','NYSE ARCA','NASDAQ', 'NYSE MKT', 'AMEX', ]
filter = df[(df.priceCurrency=='USD')].exchange.map(lambda x: 1 if x in list_of_strings else 0)

In [10]:
df1 = df[(df.priceCurrency=='USD')].dropna()
df1['filter']= filter

In [11]:
df1 = df1[(filter == 1) & (df1.endDate == '2020-11-20') & ~(df1.assetType =='ETF') & ~(df1.assetType =='Mutual Fund')]
df1

<ipython-input-11-9c754ab2a25a>:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df1 = df1[(filter == 1) & (df1.endDate == '2020-11-20') & ~(df1.assetType =='ETF') & ~(df1.assetType =='Mutual Fund')]


,ticker,exchange,assetType,priceCurrency,startDate,endDate,filter
1,-P-S,NYSE,Stock,USD,2018-08-22,2020-11-20,1
4310,A,NYSE,Stock,USD,1999-11-18,2020-11-20,1
4311,AA,NYSE,Stock,USD,2016-11-01,2020-11-20,1
4367,AACG,NASDAQ,Stock,USD,2008-01-29,2020-11-20,1
4376,AACQ,NASDAQ,Stock,USD,2020-09-04,2020-11-20,1
...,...,...,...,...,...,...,...
89115,ZYME,NYSE,Stock,USD,2017-04-28,2020-11-20,1
89116,ZYNE,NASDAQ,Stock,USD,2015-08-05,2020-11-20,1
89120,ZYXI,NASDAQ,Stock,USD,2002-12-31,2020-11-20,1
89125,ZZK,NYSE ARCA,Stock,USD,2020-07-22,2020-11-20,1


In [12]:
df1 = df1.set_index(df1.ticker).drop(index = '-P-S', columns = ['ticker','filter'])
df1

,exchange,assetType,priceCurrency,startDate,endDate
ticker,,,,,
A,NYSE,Stock,USD,1999-11-18,2020-11-20
AA,NYSE,Stock,USD,2016-11-01,2020-11-20
AACG,NASDAQ,Stock,USD,2008-01-29,2020-11-20
AACQ,NASDAQ,Stock,USD,2020-09-04,2020-11-20
AACQU,NASDAQ,Stock,USD,2020-07-14,2020-11-20
...,...,...,...,...,...
ZYME,NYSE,Stock,USD,2017-04-28,2020-11-20
ZYNE,NASDAQ,Stock,USD,2015-08-05,2020-11-20
ZYXI,NASDAQ,Stock,USD,2002-12-31,2020-11-20


In [13]:
tickers = ['AAPL', 'TSLA', 'MSFT', 'BABA']

In [14]:
stock_list = pd.DataFrame()
for ticker in df1.iloc[len(stock_list):,:].index:
    headers = {'Content-Type': 'application/json'}
    requestResponse = requests.get("https://api.tiingo.com/tiingo/daily/" + ticker + "?token=8a3dcff27b736ecb02df8d6da6db86e3d35354eb", headers=headers)
    data = pd.DataFrame.from_dict(requestResponse.json(), orient="index").transpose()
    stock_list = stock_list.append(data)

stock_list

,exchangeCode,description,ticker,startDate,name,endDate
0,NYSE,Agilent Technologies Inc. is a global leader i...,A,1999-11-18,Agilent Technologies Inc,2020-11-20
0,NYSE,"Alcoa is a global industry leader in bauxite, ...",AA,2016-11-01,Alcoa Corp,2020-11-20
0,NASDAQ,ATA Inc. (ATA) is a provides computer-based te...,AACG,2008-01-29,ATA Inc,2020-11-20
0,NASDAQ,,AACQ,2020-09-04,Artius Acquisition Inc - Class A,2020-11-20
0,NASDAQ,Artius Acquisition Inc Unit 1 Com & 13 Wt Exp ...,AACQU,2020-07-14,Artius Acquisition Inc - Units (1 Ord Share Cl...,2020-11-20
...,...,...,...,...,...,...
0,NYSE,Zymeworks is a clinical-stage biopharmaceutica...,ZYME,2017-04-28,Zymeworks Inc,2020-11-20
0,NASDAQ,Zynerba Pharmaceuticals is the leader in innov...,ZYNE,2015-08-05,Zynerba Pharmaceuticals Inc,2020-11-20
0,NASDAQ,"Zynex, founded in 1996, markets and sells its ...",ZYXI,2002-12-31,Zynex Inc,2020-11-20
0,NYSE ARCA,,ZZK,2020-07-22,None,2020-11-20


In [17]:
stock_list = stock_list.set_index(stock_list.ticker).drop(columns = 'ticker')

In [21]:
stock_list.to_csv(r'stock_description.csv')

In [1]:
import fasttext